# VacationPy

In [1]:
import pandas as pd
import numpy as np
import json
import requests
import time
from pprint import pprint 

import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import weather_api_key

import gmaps
from api_keys import g_key
gmaps.configure(api_key=g_key)

In [2]:
filepath = "cities_new.csv"
df = pd.read_csv(filepath)
df = df.drop(['Unnamed: 0'], axis=1)
df = df.rename(columns={"City Name": "City"})
df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Punta Arenas,-53.15,-70.92,46.40,81,90,8.05
1,Bengkulu,-3.80,102.27,75.29,86,100,4.03
2,Severo-Kuril'sk,50.68,156.12,38.79,73,98,31.65
3,Lillooet,50.69,-121.94,50.85,93,90,8.05
4,Tahoua,14.89,5.27,82.40,23,0,4.70


In [3]:
coords = df[["Latitude", "Longitude"]]
humids = df["Humidity"]

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity = 500)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [4]:
mask = (df["Temperature"] >= 70) & (df["Temperature"] <= 78) & (df["Wind Speed"] <= 7) & (df["Cloudiness"] == 0)
ideal_weather_cities = df.loc[mask].reset_index(drop=True)
ideal_weather_cities

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Misratah,32.38,15.09,70.70,62,0,6.64
1,Siwāna,25.63,72.42,71.60,43,0,4.52
2,San Vicente,-26.62,-54.13,73.18,31,0,6.22
3,Wānkāner,22.62,70.93,75.78,40,0,5.14
4,Luxor,25.70,32.64,71.60,53,0,2.24
5,Salalah,17.02,54.09,73.40,69,0,2.24
6,Esna,25.29,32.55,71.60,53,0,2.24
7,Gorontalo,0.54,123.06,74.79,87,0,2.15
8,Hays,30.05,-98.03,77.68,39,0,1.01
9,Emerald,-23.53,148.17,70.75,68,0,3.36


In [5]:
#inits
hotels = []
countries = []

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "distance" : 5000,
    "type": "hotel",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in ideal_weather_cities.iterrows():
    
    city_name = row["City"]
    location = f"{row.Latitude}, {row.Longitude}"

    # add location to params dict
    params["location"] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    #pprint(results)
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
    
        hotel = results[0]['name']
        country = results[0]['plus_code']['compound_code'].split(",")[1]
        
        hotels.append(hotel)
        countries.append(country)
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    time.sleep(1)
        
print("------------")

ideal_weather_cities["Hotel Name"] = hotels
ideal_weather_cities["Country"] = countries
ideal_weather_cities

Retrieving Results for Index 0: Misratah.
Closest hotel in Misratah is Mafaza Group.
Retrieving Results for Index 1: Siwāna.
Closest hotel in Siwāna is Poonam nagar siwana.
Retrieving Results for Index 2: San Vicente.
Closest hotel in San Vicente is Autodromo San Pedro.
Retrieving Results for Index 3: Wānkāner.
Closest hotel in Wānkāner is 9 avenue residency.
Retrieving Results for Index 4: Luxor.
Closest hotel in Luxor is El-Mekashkesh.
Retrieving Results for Index 5: Salalah.
Closest hotel in Salalah is المديرية العامة للماليه.
Retrieving Results for Index 6: Esna.
Closest hotel in Esna is صيدليه الدكتور بيتر منسي.
Retrieving Results for Index 7: Gorontalo.
Closest hotel in Gorontalo is Rivay.Net.
Retrieving Results for Index 8: Hays.
Closest hotel in Hays is big d mini bucking bulls.
Retrieving Results for Index 9: Emerald.
Closest hotel in Emerald is Nogoa River Bridge.
Retrieving Results for Index 10: Victoria Point.
Closest hotel in Victoria Point is Bonney Douglas Design.
Retrie

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name,Country
0,Misratah,32.38,15.09,70.70,62,0,6.64,Mafaza Group,Libya
1,Siwāna,25.63,72.42,71.60,43,0,4.52,Poonam nagar siwana,Rajasthan
2,San Vicente,-26.62,-54.13,73.18,31,0,6.22,Autodromo San Pedro,Misiones Province
3,Wānkāner,22.62,70.93,75.78,40,0,5.14,9 avenue residency,Gujarat
4,Luxor,25.70,32.64,71.60,53,0,2.24,El-Mekashkesh,Egypt
5,Salalah,17.02,54.09,73.40,69,0,2.24,المديرية العامة للماليه,Oman
6,Esna,25.29,32.55,71.60,53,0,2.24,صيدليه الدكتور بيتر منسي,Egypt
7,Gorontalo,0.54,123.06,74.79,87,0,2.15,Rivay.Net,Kota Gorontalo
8,Hays,30.05,-98.03,77.68,39,0,1.01,big d mini bucking bulls,TX
9,Emerald,-23.53,148.17,70.75,68,0,3.36,Nogoa River Bridge,Australia


In [6]:
list_info = []

for indx, row in ideal_weather_cities.iterrows():
    info = f"City: {row.City}<br>-----------<br>Hotel: {row['Hotel Name']}<br>-----------<br>Country: {row.Country}"
    list_info.append(info)

In [7]:
coords = df[["Latitude", "Longitude"]]
coords_ideal = ideal_weather_cities[["Latitude", "Longitude"]]
humids = df["Humidity"]

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coords_ideal, info_box_content=list_info)
# Add the layer to the map
fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=500)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

<Figure size 432x288 with 0 Axes>